In [0]:
!pip install pandas==0.24.2

In [0]:
!pip install scikit-learn==0.21.2

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import random

from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import ParameterGrid
import lightgbm as lgb

In [42]:
if os.name=='posix':
    # GOOGLE COLAB SETUP

    # Load the Drive helper and mount
    from google.colab import drive

    # This will prompt for authorization.
    drive.mount('/content/drive')
    
    #2. Get the file
    data_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
    codes_path    = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/AOBDL_TML'


    #3. Read file as panda dataframe
    train         = pd.read_csv(f'{data_path}/facebook_cleaned_no_punkt.csv') 
else:
        #2. Get the file
    data_path     = '../data'
    codes_path    = '../AOBDL_TML'


    #3. Read file as panda dataframe
    train         = pd.read_csv(f'{data_path}/facebook_cleaned_no_punkt.csv') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train.drop(['id', 'agr'], axis=1, inplace=True)
train.text.fillna("empty", inplace=True)

In [44]:
train[train.text.isna()]

,text,mal


In [0]:
rs=42
from sklearn.model_selection import train_test_split      
X, X_test, y, y_test  = train_test_split(train.text, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X = np.array(X)
y = np.array(y)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [47]:
print(f'X {X.shape}; X_test {X_test.shape}')

X (11300,); X_test (4616,)


In [0]:
# Ridge

In [67]:
kf  = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
fscore_ = []
c   = 0

word_vectorizer   = TfidfVectorizer(
    sublinear_tf  = True,
    strip_accents = 'unicode',
    analyzer      = 'word',
    #token_pattern = r'\w{1,}',
    #stop_words    = 'english',
    ngram_range   = (1, 1),
    max_features  = 10000)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
#auc_pr = 0.093434375824395
#auc_roc = 0.02343524395
c = 0
C_parameter = np.arange(0.1, 1.1, 0.1) 

# use best C

C_parameter = [1]

In [69]:
for c_p in C_parameter:  
      for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = LogisticRegression(C=c_p, solver='sag')
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)
            
            threshold = 0.4
            #for threshold in np.arange(0.1, 1, 0.1):
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            if len(C_parameter)==1:
                # print performance
                print(f'---------------------------------------------')
                print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
                print(f'---------------------------------------------')
                print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            fscore_.append(fscore)

      if len(C_parameter)!=1:
             print(f'PARAMETER C = {c_p}')

      # print performance
      print(f'-----------------------------------------------')
      print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
      print(f'-----------------------------------------------')
      print(f'')

 0.4 fold 0 precision 0.668 recall 0.932 fscore 0.778
---------------------------------------------
FOLD 0: AUC PR-C = 0.804, AUC ROC = 0.78
---------------------------------------------

 0.4 fold 1 precision 0.664 recall 0.944 fscore 0.78
---------------------------------------------
FOLD 1: AUC PR-C = 0.802, AUC ROC = 0.784
---------------------------------------------

 0.4 fold 2 precision 0.662 recall 0.919 fscore 0.77
---------------------------------------------
FOLD 2: AUC PR-C = 0.795, AUC ROC = 0.771
---------------------------------------------

 0.4 fold 3 precision 0.666 recall 0.935 fscore 0.778
---------------------------------------------
FOLD 3: AUC PR-C = 0.81, AUC ROC = 0.792
---------------------------------------------

 0.4 fold 4 precision 0.673 recall 0.936 fscore 0.783
---------------------------------------------
FOLD 4: AUC PR-C = 0.81, AUC ROC = 0.791
---------------------------------------------

-----------------------------------------------
CV average: 

In [70]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = LogisticRegression(C=1, solver='sag')
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 4)}, AUC ROC = {round(auc_roc, 4)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.4
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


-----------------------------------------
TEST: AUC PR-C = 0.803, AUC ROC = 0.7811
-----------------------------------------

 0.4 precision 0.666 recall 0.929 fscore 0.776


In [0]:
# RF

In [72]:
kf  = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
fscore_ = []
c   = 0

word_vectorizer   = TfidfVectorizer(
    sublinear_tf  = True,
    strip_accents = 'unicode',
    analyzer      = 'word',
    token_pattern = r'\w{1,}',
    stop_words    = 'english',
    ngram_range   = (1, 1),
    max_features  = 10000)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [73]:
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = RandomForestClassifier(n_estimators=600, max_depth=None, max_features='auto', 
                                    min_samples_split=2, verbose = True, n_jobs=20)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            
            # print performance
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            
            threshold = 0.4
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            fscore_.append(fscore)

# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
print(f'-----------------------------------------------')
print(f'')

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   15.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   38.7s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   55.4s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    1.1s finished


---------------------------------------------
FOLD 0: AUC PR-C = 0.793, AUC ROC = 0.766
---------------------------------------------

 0.4 fold 0 precision 0.67 recall 0.909 fscore 0.771


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   15.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   39.3s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   56.4s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    1.1s finished


---------------------------------------------
FOLD 1: AUC PR-C = 0.796, AUC ROC = 0.768
---------------------------------------------

 0.4 fold 1 precision 0.67 recall 0.915 fscore 0.774


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   14.8s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   38.0s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   54.4s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    1.1s finished


...

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   15.5s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   38.9s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   55.5s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    1.1s finished


---------------------------------------------
FOLD 3: AUC PR-C = 0.794, AUC ROC = 0.758
---------------------------------------------

 0.4 fold 3 precision 0.662 recall 0.903 fscore 0.764


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   15.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   38.8s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   55.5s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.8s


---------------------------------------------
FOLD 4: AUC PR-C = 0.793, AUC ROC = 0.768
---------------------------------------------

 0.4 fold 4 precision 0.671 recall 0.905 fscore 0.77
-----------------------------------------------
CV average: AUC PR-C = 0.792, AUC ROC = 0.763, FSCORE  = 0.769
-----------------------------------------------



[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    1.1s finished


In [74]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = RandomForestClassifier(n_estimators=600, max_depth=None, max_features='auto', 
                                    min_samples_split=2, verbose = True, n_jobs=20)
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.4
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   20.9s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   53.7s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:  1.3min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    1.5s


-----------------------------------------
TEST: AUC PR-C = 0.785, AUC ROC = 0.763
-----------------------------------------

 0.4 precision 0.673 recall 0.911 fscore 0.774


[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    2.2s finished


In [0]:
# SVM

In [78]:
penalty = ['l2', 'l1']
alpha = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
max_iter = [1000, 10000, 15000]

# best parameters
penalty = ['l2']
alpha = [0.0001]
max_iter = [1000] 

for p in penalty:
  for a in alpha:
    for i in max_iter:
      auc = []
      roc = []
      fscore_ = []
      c = 0
      # print performance
      print(f'-------------')
      print(f'penalty {p}')
      print(f'alpha {a}')
      print(f'max_iter {i}')
      print(f'-------------')
      
      for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = SGDClassifier(n_jobs=20, random_state=rs, loss='log', shuffle=False, 
                                    penalty=p, alpha=a, max_iter=i)
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)
            
            
            #print(f'---------------------------------------------')
            #print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            #print(f'---------------------------------------------')
            #print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            threshold = 0.4
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            #print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            fscore_.append(fscore)

      # print performance
      print(f'-----------------------------------------------')
      print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
      print(f'-----------------------------------------------')
      print(f'')

-------------
penalty l2
alpha 0.0001
max_iter 1000
-------------
-----------------------------------------------
CV average: AUC PR-C = 0.806, AUC ROC = 0.784, FSCORE  = 0.777
-----------------------------------------------



In [79]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = SGDClassifier(n_jobs=20, random_state=rs, loss='log', shuffle=False, 
                                    penalty=penalty[0], alpha=alpha[0], max_iter=max_iter[0])
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.4
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


-----------------------------------------
TEST: AUC PR-C = 0.813, AUC ROC = 0.786
-----------------------------------------

 0.4 precision 0.662 recall 0.937 fscore 0.776


In [0]:
# lightgbm

In [0]:
# muner of rounds
max_rounds = 600
stopping   = 600
verbose    = 200

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metrics':           'binary_logloss',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.8,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0.01,
    'min_child_weight':  2,
    'min_child_samples': 20,
    'silent':            True,
    'verbosity':         100,
    'learning_rate':     0.1,
    'max_depth':         7,
    'num_leaves':        70,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          20,
    'random_state':      rs,
}

In [85]:
auc = []
roc = []
fscore_ = []
for c, (train_index, val_index) in enumerate(kf.split(X, y)):

            X_train, X_val      = X[train_index], X[val_index]
            y_train, y_val      = y[train_index], y[val_index] 
            word_vectorizer.fit(X_train)
            train_word_features = word_vectorizer.transform(X_train)
            val_word_features   = word_vectorizer.transform(X_val)
            y_train             = y_train.astype('int')
            y_val               = y_val.astype('int')
            classifier          = lgb.LGBMClassifier(**lgb_params) 
            classifier.fit(train_word_features, y_train)
            probs               = classifier.predict_proba(val_word_features)[:,1]
            auc_roc             = roc_auc_score(y_val, probs)
            auc_pr              = average_precision_score(y_val, probs)

            
            # print performance
            print(f'---------------------------------------------')
            print(f'FOLD {c}: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
            print(f'---------------------------------------------')
            print(f'')

            auc.append(auc_pr)
            roc.append(auc_roc)
            threshold = 0.3
            probs_class = probs.copy()
            probs_class[probs_class >= threshold] = 1 
            probs_class[probs_class < threshold] = 0
            precision = precision_score(y_val, probs_class) 
            recall    = recall_score(y_val, probs_class)
            fscore    = f1_score(y_val, probs_class)
            print(f' {threshold} fold {c} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')

            fscore_.append(fscore)

# print performance
print(f'-----------------------------------------------')
print(f'CV average: AUC PR-C = {round(np.array(auc).mean(), 3)}, AUC ROC = {round(np.array(roc).mean(), 3)}, FSCORE  = {round(np.array(fscore_).mean(), 3)}')
print(f'-----------------------------------------------')
print(f'')

---------------------------------------------
FOLD 0: AUC PR-C = 0.783, AUC ROC = 0.744
---------------------------------------------

 0.3 fold 0 precision 0.637 recall 0.954 fscore 0.764
---------------------------------------------
FOLD 1: AUC PR-C = 0.799, AUC ROC = 0.758
---------------------------------------------

 0.3 fold 1 precision 0.627 recall 0.948 fscore 0.755
---------------------------------------------
FOLD 2: AUC PR-C = 0.776, AUC ROC = 0.739
---------------------------------------------

 0.3 fold 2 precision 0.632 recall 0.943 fscore 0.757
---------------------------------------------
FOLD 3: AUC PR-C = 0.782, AUC ROC = 0.753
---------------------------------------------

 0.3 fold 3 precision 0.638 recall 0.951 fscore 0.764
---------------------------------------------
FOLD 4: AUC PR-C = 0.781, AUC ROC = 0.74
---------------------------------------------

 0.3 fold 4 precision 0.631 recall 0.952 fscore 0.759
-----------------------------------------------
CV avera

In [86]:
# TRAIN ON WHOLE DAATA AND PREDICT ON TEST
word_vectorizer.fit(X)
train_word_features  = word_vectorizer.transform(X)
test_word_features   = word_vectorizer.transform(X_test)
classifier           = lgb.LGBMClassifier(**lgb_params) 
classifier.fit(train_word_features, y)
probs                = classifier.predict_proba(test_word_features)[:,1]
auc_roc              = roc_auc_score(y_test, probs)
auc_pr               = average_precision_score(y_test, probs)

# print performance
print(f'-----------------------------------------')
print(f'TEST: AUC PR-C = {round(auc_pr, 3)}, AUC ROC = {round(auc_roc, 3)}')
print(f'-----------------------------------------')
print(f'')

threshold = 0.3
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0
precision = precision_score(y_test, probs_class) 
recall    = recall_score(y_test, probs_class)
fscore    = f1_score(y_test, probs_class)
print(f' {threshold} precision {round(precision, 3)} recall {round(recall, 3)} fscore {round(fscore,3)}')


-----------------------------------------
TEST: AUC PR-C = 0.777, AUC ROC = 0.745
-----------------------------------------

 0.3 precision 0.63 recall 0.958 fscore 0.76
